# Policy refinement with human feedback (Policy v1 → v2)

이 노트북은 두 가지를 해준다:

1. **human_preferred_decision 컬럼이 포함된 피드백 테이블 템플릿 만들기**
   - 입력: `../results/disagreement_cases_with_text.csv`
   - 출력: `../results/disagreement_with_human_template.csv`
   - 여기에 사람이 `human_preferred_decision`을 채워 넣는다.

2. **사람 피드백 + 기존 정책 설명을 바탕으로, LLM에게 Policy A/B/C v2 초안을 요청**
   - 입력:
     - 사람이 라벨링한 `disagreement_with_human_filled.csv` (파일명은 아래에서 지정)
     - 기존 Policy A/B/C 설명 텍스트
   - 출력:
     - 터미널 출력 + `../results/policy_v2_suggestions.md` 파일


In [1]:
import os
from pathlib import Path
import pandas as pd
import textwrap

from dotenv import load_dotenv
from openai import OpenAI

# 프로젝트 디렉토리 구조 가정
PROJECT_ROOT = Path("..").resolve()
DATA_DIR = PROJECT_ROOT / "data"
RESULTS_DIR = PROJECT_ROOT / "results"

DISAGREE_CSV = RESULTS_DIR / "disagreement_cases_with_text.csv"

print("PROJECT_ROOT :", PROJECT_ROOT)
print("RESULTS_DIR  :", RESULTS_DIR)
print("DISAGREE_CSV :", DISAGREE_CSV)

# Load environment variables (expects .env with OPENAI_API_KEY)
load_dotenv()

client = OpenAI(api_key=os.getenv("OPENAI_API_KEY", ""))


PROJECT_ROOT : C:\Users\Gibeom Kim\Desktop\UnderGraduate\3. junior\techno_science_jayu
RESULTS_DIR  : C:\Users\Gibeom Kim\Desktop\UnderGraduate\3. junior\techno_science_jayu\results
DISAGREE_CSV : C:\Users\Gibeom Kim\Desktop\UnderGraduate\3. junior\techno_science_jayu\results\disagreement_cases_with_text.csv


In [3]:
# 1) 사람 피드백용 템플릿 만들기

if not DISAGREE_CSV.exists():
    raise FileNotFoundError(f"{DISAGREE_CSV} 가 없습니다. 먼저 분석 노트북을 돌려 disagreement_cases_with_text.csv를 생성하세요.")

df_dis = pd.read_csv(DISAGREE_CSV)

print("원본 disagreement_cases_with_text.csv 예시:")
display(df_dis.head())

# human_preferred_decision 컬럼 추가 (빈 값)
if "human_preferred_decision" not in df_dis.columns:
    df_dis["human_preferred_decision"] = ""

# 설명용 컬럼도 하나 추가할 수 있음 (선택)
if "human_comment" not in df_dis.columns:
    df_dis["human_comment"] = ""

TEMPLATE_PATH = RESULTS_DIR / "disagreement_with_human_template.csv"
df_dis.to_csv(TEMPLATE_PATH, index=False, encoding="utf-8-sig")

print("\n사람 라벨링 템플릿을 저장했습니다:")
print(" ->", TEMPLATE_PATH)
print("\n이 파일을 엑셀/VSCode 등으로 열어서 다음 컬럼을 채우세요:")
print(" - human_preferred_decision: BLOCK / WARN_AND_ALLOW / ALLOW 중 하나")
print(" - human_comment (선택): 왜 그렇게 판단했는지 간단 설명")


원본 disagreement_cases_with_text.csv 예시:


,sample_id,A,B,C,thread_id,role,order_in_thread,text
0,1,WARN_AND_ALLOW,WARN_AND_ALLOW,ALLOW,1,post,0,얼마나 인기없으면 글이 안올라오냐
1,21,WARN_AND_ALLOW,WARN_AND_ALLOW,ALLOW,4,comment,2,괘씸해서 안받아주지 어짜피 초과학기인데 그냥 다음학기에 들어
2,26,BLOCK,WARN_AND_ALLOW,ALLOW,5,post,0,전전 3.6졸이면 ㅁㅌㅊ임? 전공 성적은 3.9 넘고 기초랑 교양에서 많이 까먹긴함
3,30,BLOCK,BLOCK,WARN_AND_ALLOW,5,comment,4,4.1졸이면 ㅁㅌㅊ임?
4,79,WARN_AND_ALLOW,WARN_AND_ALLOW,ALLOW,10,comment,7,공군은 그냥 공익아님?



사람 라벨링 템플릿을 저장했습니다:
 -> C:\Users\Gibeom Kim\Desktop\UnderGraduate\3. junior\techno_science_jayu\results\disagreement_with_human_template.csv

이 파일을 엑셀/VSCode 등으로 열어서 다음 컬럼을 채우세요:
 - human_preferred_decision: BLOCK / WARN_AND_ALLOW / ALLOW 중 하나
 - human_comment (선택): 왜 그렇게 판단했는지 간단 설명


## 2) 사람이 할 일

1. 방금 생성된 `disagreement_with_human_template.csv` 파일을 엽니다.
2. 각 row에 대해:
   - **human_preferred_decision**: `BLOCK`, `WARN_AND_ALLOW`, `ALLOW` 중 하나를 입력
   - **human_comment**: (선택) "자기비하라서 허용", "성기 언급은 최소 경고" 등 이유를 적어도 좋음
3. 수정한 파일을 예를 들어 `disagreement_with_human_filled.csv` 같은 이름으로 저장합니다.
   - 템플릿 파일을 그대로 덮어써도 되지만,
   - 원본을 남기고 싶으면 새 파일 이름을 쓰세요.

아래 셀에서는, 사람이 채운 CSV 파일 경로를 `FILLED_PATH` 변수로 지정해서 사용합니다.


In [4]:
# 3) 사람이 채운 피드백 파일 불러오기

# 사람이 채운 파일 이름을 여기서 지정하세요.
FILLED_PATH = RESULTS_DIR / "disagreement_with_human_filled.csv"  # 예시 이름

if not FILLED_PATH.exists():
    raise FileNotFoundError(
        f"{FILLED_PATH} 가 없습니다. disagreement_with_human_template.csv를 채운 뒤, "
        "이름을 disagreement_with_human_filled.csv 로 저장하거나, 위 경로를 원하는 이름으로 바꾸세요."
    )

df_feedback = pd.read_csv(FILLED_PATH)

print("사람 피드백이 포함된 데이터 예시:")
display(df_feedback.head())

# human_preferred_decision이 비어있는 row는 제외 (필요시)
df_feedback = df_feedback[df_feedback["human_preferred_decision"].notna() & (df_feedback["human_preferred_decision"] != "")]
print(f"유효한 피드백 row 수: {len(df_feedback)}")


사람 피드백이 포함된 데이터 예시:


,sample_id,A,B,C,thread_id,role,order_in_thread,text,human_preferred_decision,human_comment
1,WARN_AND_ALLOW,WARN_AND_ALLOW,ALLOW,1,post,0,얼마나 인기없으면 글이 안올라오냐,ALLOW,ALLOW,NaN
21,WARN_AND_ALLOW,WARN_AND_ALLOW,ALLOW,4,comment,2,괘씸해서 안받아주지 어짜피 초과학기인데 그냥 다음학기에 들어,ALLOW,ALLOW,NaN
26,BLOCK,WARN_AND_ALLOW,ALLOW,5,post,0,전전 3.6졸이면 ㅁㅌㅊ임? 전공 성적은 3.9 넘고 기초랑 교양에서 많이 까먹긴함,ALLOW,ALLOW,NaN
30,BLOCK,BLOCK,WARN_AND_ALLOW,5,comment,4,4.1졸이면 ㅁㅌㅊ임?,ALLOW,ALLOW,NaN
79,WARN_AND_ALLOW,WARN_AND_ALLOW,ALLOW,10,comment,7,공군은 그냥 공익아님?,WARN_AND_ALLOW,WARN_AND_ALLOW,NaN


유효한 피드백 row 수: 25


## 4) 기존 Policy A/B/C v1 텍스트 입력

아래 셀의 `POLICY_A_V1`, `POLICY_B_V1`, `POLICY_C_V1` 문자열 안에  
**지금 시뮬레이션에 사용하고 있는 정책 설명(프롬프트용 규칙 텍스트)**를 그대로 붙여 넣으면 됩니다.

- 너무 길어도 괜찮지만, 가능하면 **사람이 읽을 수 있을 정도로 정리된 버전**이 좋습니다.
- 나중에 LLM에게는 이 텍스트와 `df_feedback`를 함께 넘겨서,
  - 정책이 어디서 과도하게 차단/허용하는지
  - 사람 피드백과 더 잘 맞도록 어떻게 수정하면 좋은지
  제안하도록 요청합니다.


In [5]:
# TODO: 여기에 현재 사용중인 A/B/C 정책 설명을 붙여 넣으세요.

POLICY_A_V1 = """Policy A: High Protection, Low Tolerance (매너 최우선)

- BLOCK if:
  - Any direct insult or derogatory expression toward a person or group
    (예: "멍청이", "병x", "저년", "틀딱" 등).
  - Swearing or profanity directed at a person, group, or their work
    (예: "씨발", "좆같은 과제", "저 교수 또 ㅈ같이 하네").
  - Sarcastic or mocking tone that targets a person’s character, ability, or identity.
  - Shaming of appearance, background, academic ability, or mental health.
  - Any hate speech or slur toward social groups (gender, region, race, sexuality, etc.).
  - Any threat, doxxing, or suggestion of harm.

- WARN_AND_ALLOW if:
  - Strong criticism of ideas, policies, or institutions without personal insult.
  - Rude words aimed at a system or policy, not at a person.

- ALLOW if:
  - Polite or neutral comments.
  - Constructive criticism with respectful language.
  - Jokes or slang that are not targeted and not hateful.

Principle: When in doubt, prioritize emotional safety and manner
even if some harsh but meaningful criticism is lost.
"""

POLICY_B_V1 = """Policy B: Balanced (매너와 토론의 균형)

- BLOCK if:
  - Direct, explicit personal attack to a specific individual using insults or slurs.
  - Hate speech toward a protected/vulnerable group (gender, sexuality, race, disability, region, etc.).
  - Threats of violence, doxxing, or calls for harassment.

- WARN_AND_ALLOW if:
  - Strong, harsh expressions that mainly target ideas, policies, classes, or systems.
  - Profanity is present but not clearly directed at a specific person.
  - Mild sarcasm but focused on a public issue (tuition, dorm policy, grading, etc.).

- ALLOW if:
  - Criticism is expressed in relatively respectful language.
  - Comment contributes arguments, experiences, or information.
  - Emotional expressions that are not direct harassment or hate speech.

Principle: When in doubt, preserve debate value if the target is
an idea or policy, but never allow direct harassment or hate speech.
"""

POLICY_C_V1 = """Policy C: Minimal Regulation, High Freedom (최소 규제, 표현의 자유 우선)

- BLOCK if:
  - Severe insult or slur naming a person or clearly identifiable small group,
    combined with clear hostility or harm.
  - Clear hate speech toward protected groups using abusive slurs.
  - Explicit threats of violence, doxxing, or incitement to harassment.

- WARN_AND_ALLOW if:
  - Harsh, emotional language with insults or profanity that is not an explicit threat
    and not targeted at a protected group.
  - Rude mockery that is uncomfortable but not clearly hateful or threatening.

- ALLOW if:
  - Almost any criticism, even if rude, as long as:
    - No explicit threats.
    - No hate speech.
    - No doxxing.

Principle: When in doubt, err on the side of freedom of expression,
and only block content that is clearly hateful or threatening.
"""

print("Policy A v1 길이:", len(POLICY_A_V1))
print("Policy B v1 길이:", len(POLICY_B_V1))
print("Policy C v1 길이:", len(POLICY_C_V1))

Policy A v1 길이: 1022
Policy B v1 길이: 909
Policy C v1 길이: 831


In [32]:
# # TODO: 여기에 현재 사용중인 A/B/C 정책 설명을 붙여 넣으세요.
# 
# POLICY_A_V1 = """[Policy A v1 설명을 여기에 붙여넣기]
# - 예: '욕설/비속어, 성적 표현, 혐오 표현 등은 거의 전부 차단한다...' 등
# """
# 
# POLICY_B_V1 = """[Policy B v1 설명을 여기에 붙여넣기]
# """
# 
# POLICY_C_V1 = """[Policy C v1 설명을 여기에 붙여넣기]
# """
# 
# print("Policy A v1 길이:", len(POLICY_A_V1))
# print("Policy B v1 길이:", len(POLICY_B_V1))
# print("Policy C v1 길이:", len(POLICY_C_V1)) 

In [6]:
# 5) LLM에 넘길 프롬프트 텍스트 구성

def build_feedback_snippet(df: pd.DataFrame, max_rows: int = 20) -> str:
    """사람 피드백 테이블을 간단한 텍스트 블록으로 변환"""
    rows = []
    subset = df.head(max_rows)
    for _, r in subset.iterrows():
        row_txt = textwrap.dedent(f"""        - sample_id: {r['sample_id']}
          role: {r['role']}, thread_id: {r['thread_id']}
          original_text: {r['text']}
          policy_decisions: A={r['A']}, B={r['B']}, C={r['C']}
          human_preferred_decision: {r['human_preferred_decision']}
          human_comment: {r.get('human_comment', '')}
        """)
        rows.append(row_txt)
    return "\n".join(rows)


feedback_block = build_feedback_snippet(df_feedback, max_rows=30)

system_prompt = """You are an assistant that helps refine content moderation policies for harmful comments/reviews.

- There are three policies: A, B, C.
- Each policy currently defines how to decide one of three outcomes:
  - BLOCK
  - WARN_AND_ALLOW
  - ALLOW

Your task:
- Read the current definitions of Policy A/B/C (v1).
- Read human feedback for some disagreement cases (human-preferred decisions).
- Propose improved Policy A/B/C (v2) descriptions so that:
  - They are still simple enough to implement with an LLM.
  - They better match the human-preferred decisions on these examples.
  - The *relative* philosophy of A/B/C is preserved, roughly:
    - A: strong protection / stricter moderation
    - B: intermediate / emphasize warnings and proportionality
    - C: more permissive / emphasize freedom of expression.

Output format:
- Use Markdown.
- Provide three subsections: 'Policy A v2', 'Policy B v2', 'Policy C v2'.
- For each policy, include:
  - A 3~6 bullet outline of the main principles.
  - Clear guidance on when to choose BLOCK / WARN_AND_ALLOW / ALLOW.
  - If helpful, 2~3 short example patterns of language that should be BLOCKed vs ALLOWed.
"""

user_prompt = f"""## Current policies (v1)

### Policy A v1
{POLICY_A_V1}

### Policy B v1
{POLICY_B_V1}

### Policy C v1
{POLICY_C_V1}

---

## Human feedback on disagreement cases

Below are some cases where policy A/B/C disagreed, and humans provided their preferred decisions.

{feedback_block}

---

Please propose refined versions 'Policy A v2', 'Policy B v2', 'Policy C v2' as described in the system instructions.
"""


print("=== System prompt preview ===\n")
print(system_prompt)

print("=== User prompt preview ===\n")
print(user_prompt)


=== System prompt preview ===

You are an assistant that helps refine content moderation policies for harmful comments/reviews.

- There are three policies: A, B, C.
- Each policy currently defines how to decide one of three outcomes:
  - BLOCK
  - WARN_AND_ALLOW
  - ALLOW

Your task:
- Read the current definitions of Policy A/B/C (v1).
- Read human feedback for some disagreement cases (human-preferred decisions).
- Propose improved Policy A/B/C (v2) descriptions so that:
  - They are still simple enough to implement with an LLM.
  - They better match the human-preferred decisions on these examples.
  - The *relative* philosophy of A/B/C is preserved, roughly:
    - A: strong protection / stricter moderation
    - B: intermediate / emphasize warnings and proportionality
    - C: more permissive / emphasize freedom of expression.

Output format:
- Use Markdown.
- Provide three subsections: 'Policy A v2', 'Policy B v2', 'Policy C v2'.
- For each policy, include:
  - A 3~6 bullet outline 

In [7]:
# 6) LLM 호출 (정책 v2 제안 받기)

response = client.chat.completions.create(
    model="gpt-5.1",  # 필요에 따라 다른 모델로 변경 가능
    messages=[
        {"role": "system", "content": system_prompt},
        {"role": "user", "content": user_prompt},
    ],
    temperature=0.3,
)

answer = response.choices[0].message.content
print(answer)

# 결과를 파일로 저장
POLICY_V2_PATH = RESULTS_DIR / "policy_v2_suggestions.md"
with open(POLICY_V2_PATH, "w", encoding="utf-8") as f:
    f.write(answer)

print("\n정책 v2 제안 내용을 저장했습니다:")
print(" ->", POLICY_V2_PATH)


### Policy A v2 – High Protection, Low Tolerance (매너 최우선)

**Main principles**

- 사람·집단에 대한 공격을 매우 엄격하게 제한한다.
- 욕설·비하 표현이 사람/집단을 겨냥하면 대부분 차단한다.
- 다만, 욕설이 전혀 없고 토론 가치가 뚜렷한 경우는 경고 없이 허용할 수 있다.
- 아이디어·정책 비판은 허용하지만, 인신공격·조롱이 섞이면 차단 또는 경고.
- 애매하면 “경고 후 허용”을 쓰고, 반복되거나 수위가 높으면 차단.

**BLOCK (차단)**

다음에 해당하면 차단:

1. **직접적인 인신공격/비하**
   - 사람이나 소규모 집단(교수 한 명, 특정 학생 등)을 향한 욕설·비하:
   - 예: “너 멍청이냐”, “그 교수 병x임”, “저년 진짜 수준 낮다”.
   - 외모·배경·학업능력·정신건강 등을 깎아내리는 표현:
     - 예: “얼굴이 역겨워서 보기 싫다”, “정신병자 아니냐”.

2. **조롱·모욕이 중심인 표현**
   - 상대의 인격·능력을 비꼬거나 모욕하는 말이 핵심인 경우:
   - 예: “니 같은 애가 뭘 알아”, “수업 태도 보니 답도 없다”.

3. **혐오 발언·차별 표현**
   - 성별, 지역, 인종, 성적지향, 장애 등 사회집단에 대한 비하/욕설:
   - 예: “틀딱들은 다 쓸모없다”, “게이들은 역겹다”.

4. **위협·신상털이·해악 조장**
   - 폭력·자해·살해·성폭력 등 구체적 위협, 신상공개, 괴롭힘 선동:
   - 예: “저 새끼 패버리고 싶다”, “저 사람 학교랑 이름 까발리자”.

**WARN_AND_ALLOW (경고 후 허용)**

다음에 해당하면 경고 문구를 붙이고 게시 허용:

1. **강한 감정 표현 + 아이디어/정책 비판**
   - 욕설·거친 표현이 있으나 **사람이 아니라 제도·정책·수업·과제**를 겨냥:
   - 예: “이 수업 커리큘럼 진짜 개판이다”, “학사 행정 진짜 ㅈ같이 운영하네”.
   - 단, 특정 개인을 직